In [1]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# Import CSV for TTVFast simulations
df = pd.read_csv(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME\validation\2nd_order_TTVFast\deckagol_TTVFast_params.csv")
df.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,m1,m2,mu1,mu2,validity1,validity2,valid
0,0.047400,100.700641,0.217648,119.317930,-0.021691,3:1,3,2,7,20.544486,1.209026,2.173868,0.000004,0.000007,331.092631,595.315253,True
1,0.164720,302.045726,0.068150,303.935202,0.017168,3:1,3,2,7,21.360529,3.932763,3.664654,0.000012,0.000011,162.568618,151.485795,True
2,0.240904,196.166685,0.042103,127.431653,0.021325,3:1,3,2,7,21.447826,3.099576,0.429561,0.000009,0.000001,1463.134849,202.771682,True
3,0.078053,285.258275,0.049500,192.127677,0.018148,3:1,3,2,7,21.381102,12.754481,4.651202,0.000038,0.000014,302.036944,110.144400,True
4,0.265350,58.254765,0.175747,166.960578,-0.016945,3:1,3,2,7,20.644161,1.778444,0.546847,0.000005,0.000002,658.806422,202.573839,True


In [2]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "std1", "res1",
               "A2", "B2", "V2", "Pttv2", "phase2", "std2", "res2",
               "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,...,res1,A2,B2,V2,Pttv2,phase2,std2,res2,R2_1,R2_2
0,0.047400,100.700641,0.217648,119.317930,-0.021691,3:1,3,2,7,20.544486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.164720,302.045726,0.068150,303.935202,0.017168,3:1,3,2,7,21.360529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.240904,196.166685,0.042103,127.431653,0.021325,3:1,3,2,7,21.447826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.078053,285.258275,0.049500,192.127677,0.018148,3:1,3,2,7,21.381102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.265350,58.254765,0.175747,166.960578,-0.016945,3:1,3,2,7,20.644161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def ttv_model(t, A, B, V, Pttv, phase):
    return A + B*t + V * np.sin(2*np.pi/Pttv * t + phase)

In [4]:
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):
    # If invalid scenario, skip
    if row['valid'] == False:
        continue
    
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']; N = row['N'] 
    PTTV = 1/np.abs(j/P2 - (j-N)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P2 * 200
    Time = begin_time   # days
    dt = P1 / 100       # days
    Total = max(250, PTTV * 2.5)  # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If few TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')
    std0, std1 = np.std(ttv0), np.std(ttv1)  # TTV dispersion
    res0, res1 = np.std(ttv0 - ttv_model(time0, *popt_ttv0)), np.std(ttv1 - ttv_model(time1, *popt_ttv1)) # Residual dispersion

    # Write results to df_fit ONLY
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, std0, res0, *popt_ttv1, std1, res1, r2_0, r2_1]

100%|████████████████████████████████████████████████████████████████████████████| 18000/18000 [28:52<00:00, 10.39it/s]


In [5]:
# If output fit has no values, convert valid from TRUE to FALSE
output_has_nan = df_fit[df_fit_cols].isna().any(axis=1)
df_fit.loc[output_has_nan, 'valid'] = False

# If output has PTTV wildly different from each other, convert valid from TRUE to FALSE
Pttv_ratio_threshold = 1.01
Pttv_not_similar = (df_fit['Pttv1'] / df_fit['Pttv2'] > Pttv_ratio_threshold) | (df_fit['Pttv2'] / df_fit['Pttv1'] > Pttv_ratio_threshold)
df_fit.loc[Pttv_not_similar, 'valid'] = False

In [6]:
# Export dataframes
# df_ttv.to_csv("deckagol_TTVFast_outputs.csv", index=False)
df_fit.to_csv("deckagol_TTVFast_fits.csv", index=False)